# Data Preparation and Transforming

#### Experimenting with data preparation for dealing with multi-dimensional data

Links:
- [Essential Commands for Data Preparation with `Pandas`](https://towardsdatascience.com/essential-commands-for-data-preparation-with-pandas-ed01579cf214)

Experimenting with some data preparation to squash daily data down to weekly data. Weekly data may work better for the (S)ARIMA(X) model (**notebook**: [SARIMAX Model](SARIMAX%20Model.ipynb)) since it'll be less volatile/granular.

In [ ]:
from dotenv import load_dotenv
import os

import pandas as pd
import matplotlib.pyplot as plt

load_dotenv()
DATASET_PATH = os.environ.get("DATASET_PATH")

df_main = pd.read_excel(DATASET_PATH + "Conversion by Day (single dimension).xlsx")
df = df_main.loc[df_main.iloc[:, 0] == "WA"]  # pick WA as an example
# resample the daily data as weekly data
weekly_data = df.resample(
        "W",
        label="right",
        closed="right",
        on="QUOTE_DATE"
    ).sum().reset_index().sort_values(by="QUOTE_DATE")
display(weekly_data)
# create a new column to store conversion rate; display data
weekly_data["Net Closing Rate"] = weekly_data["Day 1 Sales Volume"] / weekly_data["Number of New Business Quotes"]
# display data and plot it
weekly_data.head()
plt.plot(weekly_data["Net Closing Rate"]);

Experimenting with multi-dimensional data:

In [ ]:
df_main = pd.read_excel(DATASET_PATH + "Conversion by Day (multi dimension).xlsx")
df_main.head()

Merge down to the STATE_CODE dimension:

In [ ]:
# filter all other rows out
df = df_main[["QUOTE_DATE", "STATE_CODE", "QUOTE_COUNT", "SALES_COUNT"]]
df = df_main.groupby(["QUOTE_DATE", "STATE_CODE"]).aggregate({
    "QUOTE_COUNT": "sum",
    "SALES_COUNT": "sum",
}).sort_values(["STATE_CODE", "QUOTE_DATE"]).reset_index()
df.insert(2, "CONVERSION_RATE", df["SALES_COUNT"] / df["QUOTE_COUNT"])
df.head()

Merge down to the VEHICLE_KMS dimension:

In [ ]:
df = df_main[["QUOTE_DATE", "VEHICLE_KMS", "QUOTE_COUNT", "SALES_COUNT"]]
df = df_main.groupby(["QUOTE_DATE", "VEHICLE_KMS"]).aggregate({
    "QUOTE_COUNT": "sum",
    "SALES_COUNT": "sum",
}).sort_values(["VEHICLE_KMS", "QUOTE_DATE"]).reset_index()
df.insert(2, "CONVERSION_RATE", df["SALES_COUNT"] / df["QUOTE_COUNT"])
df.head()